# Scaling of Missing Persons across North America, Mexico, and South America:
---
### Ayush Sarkar | Missing Persons w/ Dynamical Systems Lab @ NYU
---

# Introduction: 
* To better understand the parallels in the number of missing persons cases in urban areas, there was an interest in investigating the scaling rates between two different countries and comparing them. 
* We considered many different South American countries, including Mexico, Columbia, Brazil, and Argentina. Despite considering so many other countries, Mexico had the most complete and similar administrative divisional structure to the United States, although Colombia did seem promising.

# Methodology: 
* We consider three main datasets in regards to developing the United States CBSA case sums and population estimates for scaling: 
  1) NamUs Missing Persons Dataset (scraped as of 07/17/2025)
  2) SEER Age-adjusted Population Estimates 1969-2022 
  3) QCEW-County-MSA-CSA Crosswalk (Contains historical crosswalks from 1990 - Dec. 2003, Dec. 2003 - Feb. 2013, Feb. 2013 - July 2023, July 2023 - Present)
*    

* From investigation, it seems as if there are few public missing persons databases available due to the sensitive nature of these cases and the family members involved. There is seemingly a federal-level database, although it is not accessible to public under normal circumstances. Thus, we consider the National Missing and Unidentified Persons System, or NamUs for short, as which was scraped as of 07/17/2025 to generate the supplemental json file included. 
  * The NamUs database contained about 25,630 entries as of 07/17/2025.
  * This cases include entries from 1902-2025

# Code: 

## Data Cleaning:

### United States NamUs Cases:

#### SEER Population Estimate Cleaning:

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv

# ============================================================
# STEP 0: SEER Historical County Population Estimates Processing
# ============================================================

def clean_and_export_population_data(input_file, output_csv_file):
    cleaned_data = []

    with open(input_file, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) < 18:
                continue
            try:
                cleaned_data.append({
                    'Year': int(line[0:4]),
                    'FIPS': line[6:11],
                    'Population': int(line[18:]) if line[18:].isdigit() else None
                })
            except Exception:
                continue

    if cleaned_data:
        with open(output_csv_file, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=cleaned_data[0].keys())
            writer.writeheader()
            writer.writerows(cleaned_data)


clean_and_export_population_data(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\source\SEER Population Estimates\us_1969_2022.19ages.adjusted.txt',
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\us_pop_by_decade.csv'
)

# ============================================================
# STEP 1: Load Inputs
# ============================================================

df_population = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\us_pop_by_decade.csv',
    dtype={'Year': int, 'FIPS': str}
)

df_cencount = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\source\NBER County Population Estimates\cencounts.csv',
    dtype=str
)

df_pop_est = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\source\2024 County Population Est\co-est2024-alldata.csv',
    dtype={'STATE': str, 'COUNTY': str},
    encoding='latin1'
)

# ============================================================
# STEP 2: Aggregate SEER + Append 2023–2024
# ============================================================

df_population = (
    df_population
    .groupby(['FIPS', 'Year'], as_index=False)
    .agg({'Population': 'sum'})
)

df_pop_est['FIPS'] = df_pop_est['STATE'] + df_pop_est['COUNTY']
df_pop_est = df_pop_est[~df_pop_est['FIPS'].str.endswith('000')]

df_2023 = df_pop_est[['FIPS', 'POPESTIMATE2023']].rename(
    columns={'POPESTIMATE2023': 'Population'}
)
df_2023['Year'] = 2023

df_2024 = df_pop_est[['FIPS', 'POPESTIMATE2024']].rename(
    columns={'POPESTIMATE2024': 'Population'}
)
df_2024['Year'] = 2024

df_population = pd.concat([df_population, df_2023, df_2024], ignore_index=True)

# ============================================================
# STEP 3: Normalize FIPS (NO corrections here)
# ============================================================

df_population['FIPS'] = df_population['FIPS'].astype(str).str.zfill(5)
df_population = df_population[
    ~df_population['FIPS'].str.match(r'^\d{2}9\d{2}$')
]

df_cencount['fips'] = df_cencount['fips'].astype(str).str.zfill(5)

# ============================================================
# STEP 4: Apply FIPS Corrections TO df_cencount (✅ FIX)
# ============================================================

fips_corrections = {
    '12025': '12086',  # Miami-Dade → Dade
}

df_cencount['fips_corrected'] = df_cencount['fips'].replace(fips_corrections)

# ============================================================
# STEP 5: Authoritative Merge
# ============================================================

df_merged = df_population.merge(
    df_cencount[['fips_corrected', 'name']],
    left_on='FIPS',
    right_on='fips_corrected',
    how='left'
).drop(columns='fips_corrected')

df_merged['source'] = np.where(df_merged['name'].notna(), 'table', None)

# ============================================================
# STEP 6: Shapefile Fallback (only unresolved)
# ============================================================

df_nan = df_merged[df_merged['name'].isna()].copy()
df_nan['name_filled'] = None
df_nan['source'] = None

def build_fips_map(gdf):
    if 'GEOID' in gdf.columns:
        fips_col = 'GEOID'
    elif 'STATEFP' in gdf.columns and 'COUNTYFP' in gdf.columns:
        gdf['FIPS'] = gdf['STATEFP'].astype(str).str.zfill(2) + gdf['COUNTYFP'].astype(str).str.zfill(3)
        fips_col = 'FIPS'
    elif 'CNTY_FIPS' in gdf.columns:
        gdf['FIPS'] = gdf['CNTY_FIPS'].astype(str).str.zfill(5)
        fips_col = 'FIPS'
    else:
        raise ValueError("No recognizable FIPS columns found.")

    for name_col in ['NAMELSAD', 'NAME', 'COUNTYNAME']:
        if name_col in gdf.columns:
            gdf[name_col] = gdf[name_col].astype(str).str.strip()
            return dict(zip(gdf[fips_col].astype(str).str.zfill(5), gdf[name_col]))

    raise ValueError("No recognizable county name column found.")

county_shape_files = {
    2024: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2024\counties\tl_2024_us_county.shp',
    2023: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2023\US_county_2023.shp',
    2022: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2022\US_county_2022.shp',
    2010: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2010\US_county_2010.shp',
    2000: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2000\US_county_2000.shp',
    1990: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1990\US_county_1990.shp',
    1980: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1980\US_county_1980.shp',
    1970: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1970\US_county_1970_conflated.shp',
    1960: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1960\US_county_1960_conflated.shp',
    1950: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1950\US_county_1950_conflated.shp',
    1940: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1940\US_county_1940_conflated.shp',
    1930: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1930\US_county_1930_conflated.shp',
    1920: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1920\US_county_1920_conflated.shp',
    1910: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1910\US_county_1910_conflated.shp',
    1900: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1900\US_county_1900_conflated.shp'
}
subdivision_shape_files = {
    2023: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2023\subdivisions\US_cty_sub_2023.shp',
    2022: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2022\subdivisons\US_cty_sub_2022.shp',
    2010: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2010\subdivisions\US_cty_sub_2010.shp',
    2000: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\2000\subdivisions\US_cty_sub_2000.shp',
    1990: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1990\subdivisions\US_cty_sub_1990.shp',
    1980: r'F:\dsl_CLIMA\projects\Missing Persons Project\shape files\1980\subdivisions\US_mcd_1980.shp'
}

for year, path in county_shape_files.items():
    gdf = gpd.read_file(path)
    fips_map = build_fips_map(gdf)

    mask = df_nan['name_filled'].isna()
    matches = df_nan.loc[mask, 'FIPS'].map(fips_map)

    df_nan.loc[mask, 'name_filled'] = matches
    df_nan.loc[mask & matches.notna(), 'source'] = f'shapefile_{year}'

for year, path in subdivision_shape_files.items():
    gdf = gpd.read_file(path)
    fips_map = build_fips_map(gdf)

    mask = df_nan['name_filled'].isna()
    matches = df_nan.loc[mask, 'FIPS'].map(fips_map)

    df_nan.loc[mask, 'name_filled'] = matches
    df_nan.loc[mask & matches.notna(), 'source'] = f'shapefile_{year}'

df_nan['name'] = df_nan['name_filled']
df_merged.update(df_nan[['FIPS', 'name', 'source']])

df_merged['State'] = df_merged['name'].str.extract(r'^([A-Z]{2})\s+')

us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

state_fips_to_abbr = {
    '01':'AL','02':'AK','04':'AZ','05':'AR','06':'CA','08':'CO','09':'CT','10':'DE',
    '11':'DC','12':'FL','13':'GA','15':'HI','16':'ID','17':'IL','18':'IN','19':'IA',
    '20':'KS','21':'KY','22':'LA','23':'ME','24':'MD','25':'MA','26':'MI','27':'MN',
    '28':'MS','29':'MO','30':'MT','31':'NE','32':'NV','33':'NH','34':'NJ','35':'NM',
    '36':'NY','37':'NC','38':'ND','39':'OH','40':'OK','41':'OR','42':'PA','44':'RI',
    '45':'SC','46':'SD','47':'TN','48':'TX','49':'UT','50':'VT','51':'VA','53':'WA',
    '54':'WV','55':'WI','56':'WY'
}

df_merged['state_abbr'] = df_merged['FIPS'].str[:2].map(state_fips_to_abbr)
df_merged['State'] = df_merged['state_abbr'].map(us_state_abbrev)
df_merged = df_merged.drop(columns=['state_abbr'])
df_merged['name'] = df_merged['name'].str.replace(r'^[A-Z]{2}\s+', '', regex=True)

df_merged.loc[(df_merged['name'] == 'Dade County') & (df_merged['State'] == 'Florida'), 'name'] = 'Miami-Dade County'
df_merged.loc[(df_merged['name'] == 'La Salle County') & (df_merged['State'] == 'Illinois'), 'name'] = 'Lasalle County'
df_merged.loc[(df_merged['name'] == 'DeBaca County') & (df_merged['State'] == 'New Mexico'), 'name'] = 'De Baca County'
df_merged.loc[(df_merged['name'] == 'St. John the Baptist Par.') & (df_merged['State'] == 'Louisiana'), 'name'] = 'St. John the Baptist Parish'
df_merged.loc[(df_merged['name'] == 'Dona Ana County') & (df_merged['State'] == 'New Mexico'), 'name'] = 'DOÑA ANA COUNTY'

# ============================================================
# STEP 7: Final Export
# ============================================================

df_merged.to_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\population.csv',
    index=False
)

# print("✅ Export complete")
# print(df_merged['source'].value_counts(dropna=False))
print(df_merged.isna().sum())

#### NamUs Scraping and Cleaning:

In [ ]:
import pandas as pd
import numpy as np
import csv
import json

# ===============================
# Helper: tokenize missing values
# ===============================
def tokenize(value):
    """Convert empty/missing/unknown/redacted values into consistent tokens."""
    if value is None:
        return "MISSING"
    if isinstance(value, str):
        stripped = value.strip().lower()
        if stripped in ["", "na", "n/a", "null", "not available"]:
            return "CENSORED"
        if stripped in ["unknown", "unk"]:
            return "UNKNOWN"
    return value


# ===============================
# Load raw NamUs JSON
# ===============================
with open(
    r'F:\dsl_CLIMA\projects\Missing Persons Project\output\namus-20250717.json',
    'r',
    encoding='utf-8'
) as f:
    data = json.load(f)

main_data = []

for entry in data:
    subject = entry.get("subjectIdentification", {})
    desc = entry.get("subjectDescription", {})
    physical = entry.get("physicalDescription", {})
    sighting = entry.get("sighting", {})
    agency = entry.get("primaryInvestigatingAgency", {})

    row = {
        "CaseID": tokenize(entry.get("idFormatted")),
        "CurrentMinAge": tokenize(subject.get("currentMinAge")),
        "CurrentMaxAge": tokenize(subject.get("currentMaxAge")),
        "Sex": tokenize(desc.get("sex", {}).get("name") if desc.get("sex") else None),
        "Ethnicity": tokenize(desc.get("primaryEthnicity", {}).get("name") if desc.get("primaryEthnicity") else None),
        "DisappearanceDate": tokenize(sighting.get("date")),
        "City": tokenize(sighting.get("address", {}).get("city") if sighting.get("address") else None),
        "State": tokenize(
            sighting.get("address", {})
            .get("state", {})
            .get("name") if sighting.get("address") else None
        ),
        "County": tokenize(
            sighting.get("address", {})
            .get("county", {})
            .get("name") if sighting.get("address") else None
        ),
        "InvestigatingAgency": tokenize(agency.get("name")),
    }

    main_data.append(row)


# ===============================
# Write intermediate CSV
# ===============================
output_csv = r'F:\dsl_CLIMA\projects\submittable\missing persons\export\cleaned_missing_persons.csv'

with open(output_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=main_data[0].keys())
    writer.writeheader()
    writer.writerows(main_data)


# ===============================
# Reload as DataFrame
# ===============================
df_namus = pd.read_csv(
    output_csv,
    parse_dates=['DisappearanceDate'],
    date_parser=lambda x: pd.to_datetime(x, errors='coerce')
)

df_namus = df_namus[
    ["CaseID", "CurrentMinAge", "CurrentMaxAge", "Sex", "Ethnicity",
     "DisappearanceDate", "City", "State", "County"]
].copy()


# ===============================
# Year handling (cap pre-1969)
# ===============================
df_namus['Year'] = df_namus['DisappearanceDate'].dt.year
df_namus.loc[df_namus['Year'] < 1969, 'Year'] = 1969
df_namus.loc[df_namus['Year'] > 2024, 'Year'] = 2024
df_namus['Year'] = df_namus['Year'].astype(int)


# ===============================
# Connecticut post-2022 handling
# ===============================
connecticut_cities_to_county = {
    'EAST HARTFORD': 'CAPITOL PLANNING REGION',
    'MERIDEN': 'SOUTH CENTRAL CONNECTICUT PLANNING REGION',
    'NEW BRITAIN': 'CAPITOL PLANNING REGION',
    'TORRINGTON': 'NORTHWEST HILLS PLANNING REGION',
    'WEST HARTFORD': 'CAPITOL PLANNING REGION',
    'GLASTONBURY': 'CAPITOL PLANNING REGION',
    'DERBY': 'NAUGATUCK VALLEY PLANNING REGION',
    'LISBON': 'SOUTHEASTERN CONNECTICUT PLANNING REGION',
    'AVON': 'CAPITOL PLANNING REGION',
    'GUILFORD': 'SOUTH CENTRAL CONNECTICUT PLANNING REGION',
    'HAMDEN': 'SOUTH CENTRAL CONNECTICUT PLANNING REGION',
    'GROTON': 'SOUTHEASTERN CONNECTICUT PLANNING REGION',
    'BRIDGEPORT': 'GREATER BRIDGEPORT PLANNING REGION',
    'NEW HAVEN': 'SOUTH CENTRAL CONNECTICUT PLANNING REGION',
    'HARTFORD': 'CAPITOL PLANNING REGION',
    'LEDYARD': 'SOUTHEASTERN CONNECTICUT PLANNING REGION',
    'DANBURY': 'SOUTHEASTERN CONNECTICUT PLANNING REGION'
}

# Normalize early
df_namus['State'] = df_namus['State'].astype(str).str.strip().str.upper()
df_namus['County'] = df_namus['County'].astype(str).str.strip().str.upper()
df_namus['City'] = df_namus['City'].astype(str).str.strip().str.upper()

ct_mask = (df_namus['State'] == 'CONNECTICUT') & (df_namus['Year'] > 2022)
mapped_ct = df_namus.loc[ct_mask, 'City'].map(connecticut_cities_to_county)
df_namus.loc[ct_mask, 'County'] = mapped_ct.combine_first(df_namus.loc[ct_mask, 'County'])


# ===============================
# Drop territories
# 
# ===============================
dropped_states = {
    'PUERTO RICO',
    'VIRGIN ISLANDS',
    'GUAM',
    'NORTHERN MARIANA ISLANDS'
}
df_namus = df_namus[~df_namus['State'].isin(dropped_states)]

# ===============================
# Bad county flag
# ===============================
bad_values = {'MISSING', 'UNKNOWN', 'CENSORED'}

df_namus['County'] = (
    df_namus['County']
        .astype(str)
        .str.strip()
        .replace({'NAN': np.nan})
)

df_namus = df_namus[
    df_namus['County'].notna() &
    (~df_namus['County'].isin(bad_values))
].copy()


# ===============================
# Export final NamUs cases
# Total Cases: 25532
# ===============================
df_namus.to_csv( r'F:\dsl_CLIMA\projects\submittable\missing persons\export\namus_cases.csv', index=False)

print("Final row count:", len(df_namus))
print(df_namus)

#### Crosswalk Cleaning and Merge:

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

# --- Load files ---
df_population = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\population.csv',
    dtype={'FIPS': str}
)
df_namus = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\namus_cases.csv'
)
crosswalk_file = r'F:\dsl_CLIMA\projects\Missing Persons Project\working_dfs\qcew-county-msa-csa-crosswalk.xlsx'

# --- Helper functions ---
bad_values = {'MISSING', 'UNKNOWN', 'CENSORED'}
def clean_crosswalk(df_cw):
    df_cw[['County Name', 'State Full']] = df_cw['County Title'].str.upper().str.split(',', n=1, expand=True)
    df_cw['County Name'] = df_cw['County Name'].str.strip()
    df_cw['State Full'] = df_cw['State Full'].str.strip()
    
    msa_split = df_cw['MSA Title'].str.upper().str.split(',', n=1, expand=True)
    df_cw['MSA Name'] = msa_split[0].str.strip()
    df_cw['MSA State Abbr'] = msa_split[1].str.strip().str.slice(0, 2)
    
    df_cw['County Code'] = df_cw['County Code'].astype(str).str.zfill(5)
    df_cw['MSA Code'] = df_cw['MSA Code'].astype(str).str.zfill(5)
    
    return df_cw

def merge_pop_with_crosswalk(df_subset, cw, bad_values={'MISSING', 'UNKNOWN', 'CENSORED'}):
    df = df_subset.copy()

    # --- Normalize columns ---
    df['County_norm'] = df['County'].astype(str).str.upper().str.strip()
    df['State_norm'] = df['State'].astype(str).str.upper().str.strip()

    cw['County_norm'] = cw['County Title'].astype(str).str.upper().str.strip()
    cw['MSA_Title_norm'] = cw['MSA Title'].astype(str).str.upper().str.split(',', n=1).str[0].str.strip()
    cw['State_abbr'] = cw['MSA Title'].astype(str).str.extract(r',\s*([^\s]+)', expand=False)
    
    # Map abbreviation to full state name if needed
    us_state_abbrev = {
        'AL': 'Alabama','AK': 'Alaska','AZ': 'Arizona','AR': 'Arkansas','CA': 'California',
        'CO': 'Colorado','CT': 'Connecticut','DE': 'Delaware','FL': 'Florida','GA': 'Georgia',
        'HI': 'Hawaii','ID': 'Idaho','IL': 'Illinois','IN': 'Indiana','IA': 'Iowa','KS': 'Kansas',
        'KY': 'Kentucky','LA': 'Louisiana','ME': 'Maine','MD': 'Maryland','MA': 'Massachusetts',
        'MI': 'Michigan','MN': 'Minnesota','MS': 'Mississippi','MO': 'Missouri','MT': 'Montana',
        'NE': 'Nebraska','NV': 'Nevada','NH': 'New Hampshire','NJ': 'New Jersey','NM': 'New Mexico',
        'NY': 'New York','NC': 'North Carolina','ND': 'North Dakota','OH': 'Ohio','OK': 'Oklahoma',
        'OR': 'Oregon','PA': 'Pennsylvania','RI': 'Rhode Island','SC': 'South Carolina',
        'SD': 'South Dakota','TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VT': 'Vermont',
        'VA': 'Virginia','WA': 'Washington','WV': 'West Virginia','WI': 'Wisconsin','WY': 'Wyoming',
        'D.C.': 'District of Columbia'
    }
    cw['State_full'] = cw['State_abbr'].map(us_state_abbrev)

    # --- Split good vs bad counties ---
    good_mask = df['County'].notna() & (~df['County'].isin(bad_values))
    df_good = df[good_mask].copy()

    # --- Merge good counties by FIPS ---
    df_good = df_good.merge(
        cw[['County Code','County Title','MSA Code','CSA Code','MSA Title','CSA Title']],
        left_on='FIPS',
        right_on='County Code',
        how='left'
    ).drop(columns=['County Code'], errors='ignore')

    return df_good

def merge_cases_with_crosswalk(df_subset, cw, bad_values={'MISSING', 'UNKNOWN', 'CENSORED'}):
    df = df_subset.copy()

    # --- Normalize columns ---
    df['County_norm'] = df['County'].astype(str).str.upper().str.strip()
    df['City_norm'] = df['City'].astype(str).str.upper().str.strip() 
    df['State_norm'] = df['State'].astype(str).str.upper().str.strip()

    cw['County_norm'] = cw['County Title'].astype(str).str.upper().str.strip()
    cw['MSA_Title_norm'] = cw['MSA Title'].astype(str).str.upper().str.split(',', n=1).str[0].str.strip()
    cw['State_abbr'] = cw['MSA Title'].astype(str).str.extract(r',\s*([^\s]+)', expand=False)
    
    # Map abbreviation to full state name if needed
    us_state_abbrev = {
        'AL': 'Alabama','AK': 'Alaska','AZ': 'Arizona','AR': 'Arkansas','CA': 'California',
        'CO': 'Colorado','CT': 'Connecticut','DE': 'Delaware','FL': 'Florida','GA': 'Georgia',
        'HI': 'Hawaii','ID': 'Idaho','IL': 'Illinois','IN': 'Indiana','IA': 'Iowa','KS': 'Kansas',
        'KY': 'Kentucky','LA': 'Louisiana','ME': 'Maine','MD': 'Maryland','MA': 'Massachusetts',
        'MI': 'Michigan','MN': 'Minnesota','MS': 'Mississippi','MO': 'Missouri','MT': 'Montana',
        'NE': 'Nebraska','NV': 'Nevada','NH': 'New Hampshire','NJ': 'New Jersey','NM': 'New Mexico',
        'NY': 'New York','NC': 'North Carolina','ND': 'North Dakota','OH': 'Ohio','OK': 'Oklahoma',
        'OR': 'Oregon','PA': 'Pennsylvania','RI': 'Rhode Island','SC': 'South Carolina',
        'SD': 'South Dakota','TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VT': 'Vermont',
        'VA': 'Virginia','WA': 'Washington','WV': 'West Virginia','WI': 'Wisconsin','WY': 'Wyoming',
        'D.C.': 'District of Columbia'
    }
    cw['State_full'] = cw['State_abbr'].map(us_state_abbrev)

    # --- Split good vs bad counties ---
    good_mask = df['County'].notna() & (~df['County'].isin(bad_values))
    df_good = df[good_mask].copy()
    df_bad = df[~good_mask].copy()

    # --- Merge good counties by FIPS ---
    df_good = df_good.merge(
        cw[['County Code','County Title','MSA Code','CSA Code','MSA Title','CSA Title']],
        left_on='FIPS',
        right_on='County Code',
        how='left'
    ).drop(columns=['County Code'], errors='ignore')

    # --- Merge bad counties by City → MSA ---
    df_bad = df_bad.merge(
        cw[['MSA_Title_norm','State_full','MSA Code','CSA Code','MSA Title','CSA Title']],
        left_on=['City_norm','State_norm'],
        right_on=['MSA_Title_norm','State_full'],
        how='left'
    ).drop(columns=['MSA_Title_norm','State_full'], errors='ignore')

    # --- Fill FIPS and County for MSAs with a single county ---
    msa_single = (
        cw.groupby('MSA Code', as_index=False)
        .agg({'County Code':'nunique','County Title':'first'})
        .query('`County Code` == 1')
        .rename(columns={'County Title':'Single_County_Title'})
    )
    msa_code_fill = cw.groupby('MSA Code', as_index=False).agg({'County Code':'first'})
    msa_single = msa_single.drop(columns=['County Code']).merge(msa_code_fill, on='MSA Code')

    df_bad = df_bad.merge(msa_single, on='MSA Code', how='left')
    df_bad['FIPS'] = df_bad['FIPS'].fillna(df_bad['County Code'])
    df_bad['County'] = df_bad['County'].fillna(df_bad['Single_County_Title'])
    df_bad.drop(columns=['County Code','Single_County_Title'], inplace=True)

    # --- Recombine ---
    df_merged = pd.concat([df_good, df_bad], ignore_index=True)
    df_merged.drop(columns=['County_norm','City_norm','State_norm'], errors='ignore', inplace=True)

    return df_merged


def summarize_population_by_msa_all_years(df):
    return (
        df.groupby(['Year', 'MSA Code'], as_index=False)
          .agg(MSA_pop=('Population', 'sum'))
          .sort_values(['Year', 'MSA Code'])
          .reset_index(drop=True)
    )

def summarize_population_by_csa_all_years(df):
    return (
        df.groupby(['Year', 'CSA Code'], as_index=False)
          .agg(CSA_pop=('Population', 'sum'))
          .sort_values(['Year', 'CSA Code'])
          .reset_index(drop=True)
    )

def simplify_titles(df):
    if 'MSA Title' in df.columns:
        df['CBSA Type'] = df['MSA Title'].astype(str).str.extract(r'(\w+)$')[0].replace('nan', np.nan)
        df['MSA Title'] = df['MSA Title'].astype(str).str.split(',', n=1).str[0].str.strip()
    if 'CSA Title' in df.columns:
        df['CSA Type'] = df['CSA Title'].astype(str).str.extract(r'(\w+)$')[0].replace('nan', np.nan)
        df['CSA Title'] = df['CSA Title'].astype(str).str.split(',', n=1).str[0].str.strip()
    return df

# --- Normalize for merging ---
df_namus['County_norm'] = df_namus['County'].str.upper().str.strip()
df_namus['State_norm'] = df_namus['State'].str.upper().str.strip()
df_population['County_norm'] = df_population['name'].str.upper().str.strip()
df_population['State_norm'] = df_population['State'].str.upper().str.strip()
df_namus['Year'] = df_namus['Year'].astype(int)
df_population['Year'] = df_population['Year'].astype(int)

# --- Deduplicate population to avoid row multiplication ---
df_population = df_population.drop_duplicates(subset=['Year', 'State_norm', 'County_norm'])

# --- Load and clean crosswalks ---
cw_2003 = clean_crosswalk(pd.read_excel(crosswalk_file, sheet_name='Dec. 2003 Crosswalk', dtype=str))
cw_2013 = clean_crosswalk(pd.read_excel(crosswalk_file, sheet_name='Feb. 2013 Crosswalk', dtype=str))
cw_2023 = clean_crosswalk(pd.read_excel(crosswalk_file, sheet_name='Jul. 2023 Crosswalk', dtype=str))

# --- Split Population by year ---
df_population['County'] = df_population['name'].copy()
df_pop_2003 = df_population[df_population['Year'] <= 2003]
df_pop_2013 = df_population[(df_population['Year'] > 2003) & (df_population['Year'] < 2013)]
df_pop_2023 = df_population[df_population['Year'] >= 2013]

df_pop_final = pd.concat([
    merge_pop_with_crosswalk(df_pop_2003, cw_2003),
    merge_pop_with_crosswalk(df_pop_2013, cw_2013),
    merge_pop_with_crosswalk(df_pop_2023, cw_2023)
], ignore_index=True)

df_cbsa = summarize_population_by_msa_all_years(df_pop_final)
df_csa = summarize_population_by_csa_all_years(df_pop_final)

# --- Summarize populations ---
df_pop_final = (
    df_pop_final
    .merge(df_cbsa, on=['Year', 'MSA Code'], how='left')
    .merge(df_csa, on=['Year', 'CSA Code'], how='left')
    .rename(columns={'Population': 'County_pop'})
).copy()

df_pop_final = simplify_titles(df_pop_final)

# --- Merge population ---
df_namus = df_namus.merge(
    df_pop_final[['FIPS', 'Year', 'County_pop', 'name', 'source', 'State', 'MSA Code', 'CSA Code', 'MSA Title', 'CSA Title', 'MSA_pop', 'CSA_pop', 'CBSA Type', 'CSA Type', 'County_norm', 'State_norm']],
    on=['Year', 'County_norm', 'State_norm'],
    how='left'
).drop_duplicates()

df_namus = df_namus[['CaseID','CurrentMinAge','CurrentMaxAge','Sex','Ethnicity','DisappearanceDate','City','State_x','County','Year','FIPS','County_pop','MSA Code','CSA Code','MSA Title','CSA Title','MSA_pop','CSA_pop','CBSA Type','CSA Type']]
df_namus = df_namus.rename(columns={'State_x': 'State'})
# --- Filter years and drop territories ---
# df_namus = df_namus[(df_namus['Year'] > 1999) & (df_namus['Year'] < 2025)]

# # --- Drop rows without FIPS after merge ---
df_namus = df_namus[df_namus['FIPS'].notna()].copy()

# --- Export ---
df_namus.to_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv', index=False)

df_pop_final = df_pop_final[['FIPS', 'Year', 'County_pop', 'name', 'source', 'State', 'MSA Code', 'CSA Code', 'MSA Title', 'CSA Title', 'MSA_pop', 'CSA_pop', 'CBSA Type', 'CSA Type']]
df_pop_final.to_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\export\pop_term.csv', index=False)

print("Final row count:", len(df_namus))
print(df_namus.isna().sum())

### Mexico INEGI Cases:

In [ ]:
import pandas as pd 
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt
import missingno as msno
from datetime import datetime

df_inegi = pd.read_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\source\mexico_missing_persons\data.csv', dtype=str)

def print_unknown_confidential_counts(df):
    for col in df.columns:
        count = df[col].isin(['UNKNOWN', 'CONFIDENTIAL', 'MISSING']).sum()
        total = df[col].size
        percent = (count / total) * 100 if total > 0 else 0
        print(f"{col}: (Count: {count}, Total: {total}); {percent:.2f}%")

print_unknown_confidential_counts(df_inegi)
# VICTIM_ID: (Count: 0, Total: 129830); 0.00%
# ORIGIN_AGENCY: (Count: 0, Total: 129830); 0.00%
# DATE_OF_BIRTH: (Count: 76997, Total: 129830); 59.31%
# SEX: (Count: 48105, Total: 129830); 37.05%
# DATE_OF_INCIDENCE: (Count: 55991, Total: 129830); 43.13%
# DATE_OF_REPORT: (Count: 53904, Total: 129830); 41.52%
# VICTIM_STATUS: (Count: 124808, Total: 129830); 96.13%
# STATE_ID: (Count: 0, Total: 129830); 0.00%
# STATE: (Count: 2936, Total: 129830); 2.26%
# MUNICIPALITY_ID: (Count: 0, Total: 129830); 0.00%
# MUNICIPALITY: (Count: 53132, Total: 129830); 40.92%

def plot_missing_value_correlation(df, replace_special=True, special_values=None):

    # Replace special placeholder strings with np.nan
    if replace_special:
        if special_values is None:
            special_values = ['UNKNOWN']
        df = df.replace(special_values, np.nan)

    # Plot missing value correlation heatmap
    msno.heatmap(df)
    plt.title("Missing Value Correlation Heatmap", fontsize=24)
    plt.tight_layout()
    plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\mexico\missingValue_correlation_matrix.png', dpi=1200, bbox_inches='tight')
    plt.show()

plot_missing_value_correlation(df_inegi)

df_temp = df_inegi.copy()
df_temp = df_temp[df_temp['STATE'] == 'UNKNOWN']
# print(df_temp)

def plot_valid_entries_choropleth_shp(
    df,
    state_col='STATE',
    columns_to_check=None,
    special_values=None,
    shapefile_path=None,
    shapefile_state_col='name'
):
    """
    Creates a static choropleth map of Mexican states showing number of valid (non-missing) entries.

    Parameters:
    - df (pd.DataFrame): Input dataset
    - state_col (str): Column in df identifying states (e.g. 'STATE')
    - columns_to_check (list): Columns to check for valid entries (non-'UNKNOWN')
    - special_values (list): List of strings to treat as missing (default: ['UNKNOWN', 'CONFIDENTIAL'])
    - shapefile_path (str): Path to the .shp file for Mexican state boundaries
    - shapefile_state_col (str): Column in shapefile with state names to match against df[state_col]
    """

    if shapefile_path is None:
        raise ValueError("Please provide the path to a .shp file (shapefile_path).")

    if special_values is None:
        special_values = ['UNKNOWN', 'CONFIDENTIAL']
    if columns_to_check is None:
        columns_to_check = [col for col in df.columns if col != state_col]

    df = df.copy()
    df[state_col] = df[state_col].astype(str).str.upper()

    # Replace special values with NA
    df[columns_to_check] = df[columns_to_check].replace(special_values, pd.NA)

    # Count valid entries per state
    valid_counts = df.groupby(state_col)[columns_to_check].apply(lambda g: g.notna().sum().sum()).reset_index()
    valid_counts.columns = [state_col, 'valid_count']

    # Load shapefile
    gdf = gpd.read_file(shapefile_path)
    gdf[shapefile_state_col] = gdf[shapefile_state_col].astype(str).str.upper()

    # Merge with shapefile
    merged = gdf.merge(valid_counts, left_on=shapefile_state_col, right_on=state_col, how='left')
    merged['valid_count'] = merged['valid_count'].fillna(0)

    # Plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    merged.plot(column='valid_count',
                ax=ax,
                legend=True,
                cmap='viridis',
                edgecolor='black',
                legend_kwds={'label': "Valid Entry Count", 'orientation': "vertical"})

    ax.set_title("Valid Entries by Mexican State", fontsize=15)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\mexico\cases_choropleth.png', dpi=1200, bbox_inches='tight')
    plt.show()

plot_valid_entries_choropleth_shp(
    df=df_inegi,
    state_col='STATE',
    columns_to_check=[
        'DATE_OF_BIRTH',
        'SEX',
        'DATE_OF_INCIDENCE',
        'DATE_OF_REPORT',
        'VICTIM_STATUS'
    ],
    special_values=['UNKNOWN'],
    shapefile_path=r'F:\dsl_CLIMA\projects\submittable\missing persons\source\shape files\mexico\mexican-states.shp'
)

# Data Visualization:

## NamUs Visuals:

### Scaling Plots:

##### Scaling of Cumulative Yearly NamUs Cases vs Population for Select Time Periods:

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import statsmodels.api as sm

df_primary = pd.read_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv')

# --- Parse and clean the disappearance date
df_primary['DisappearanceDate'] = pd.to_datetime(df_primary['DisappearanceDate'], errors='coerce')

# --- Set date index and sort
df_primary = df_primary.set_index('DisappearanceDate').sort_index()
print(df_primary.isna().sum())

# --- Resample: monthly counts from the full data (up to 2024)
all_months = pd.date_range(start='2010-01-01', end='2024-12-31', freq='MS')
monthly_counts = df_primary.resample('MS').size().reindex(all_months, fill_value=0)

# --- Compute cumulative disappearances
cumulative_disappearances = monthly_counts.cumsum()

# --- Filter to plot only from 2000 onward
plot_start = '2010-01-01'
cumulative_to_plot = cumulative_disappearances[cumulative_disappearances.index >= plot_start]

# --- January data points for markers (from 2000 onward)
january_points = cumulative_to_plot[cumulative_to_plot.index.month == 1]

# --- Plot
plt.figure(figsize=(14, 10))
plt.plot(cumulative_to_plot.index, cumulative_to_plot.values,
         color='darkgreen', linewidth=2, label='Cumulative NamUS Missing Persons Cases (2010–2024)')


# --- Formatting
plt.title('Cumulative NamUS Missing Persons Cases by Month Start (2010–2024)', fontsize=24)
plt.xlabel('Year', fontsize=20)
plt.ylabel('Cumulative NamUS Missing Persons Cases', fontsize=28)
plt.grid(False)

# Format x-axis ticks for Januarys only
jan_ticks = january_points.index
plt.xticks(ticks=jan_ticks, labels=[date.strftime('%Y') for date in jan_ticks], fontsize=18, rotation=55)
y_ticks=[0, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000]
plt.yticks(y_ticks, fontsize=14)
plt.ylim(y_ticks[0] + .5)
plt.xlim(all_months[0], all_months[-1])

plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\temporal\[2010-2024]cumulative_cases.png', dpi=1200, bbox_inches='tight')
plt.show()


df = df_primary.copy()

# Full year range for cumulative data (1900 to 2024 inclusive)
years = list(range(2010, 2025))  # 2025 excluded
plot_start_year = 2010
years_to_plot = []  # store years for which regression results will be plotted

# Storage for regression results
betas = []
beta_err_lower = []
beta_err_upper = []
intercepts = []
intercept_err_lower = []
intercept_err_upper = []
r2_values = []

# New storage for total population and cumulative cases
total_populations = []
cumulative_cases = []

for year in years:
    # Use all cases up to current year (cumulative)
    df_year = df[df['Year'] <= year]

    grouped = (
        df_year.groupby(['State', 'FIPS', 'County_pop'])
        .agg(case_count=('CaseID', 'count'))
        .reset_index()
    )
    grouped = grouped[grouped['case_count'] > 0]

    if len(grouped) > 1:
        X_log = np.log10(grouped['County_pop'].values)
        y_log = np.log10(grouped['case_count'].values)
        X_log_const = sm.add_constant(X_log)
        model = sm.OLS(y_log, X_log_const).fit()

        intercept = model.params[0]
        beta = model.params[1]
        r2 = model.rsquared

        conf_int = model.conf_int()
        intercept_ci_lower, intercept_ci_upper = conf_int[0, 0], conf_int[0, 1]
        beta_ci_lower, beta_ci_upper = conf_int[1, 0], conf_int[1, 1]

        # Calculate total population and cumulative cases for this year
        total_pop = grouped['County_pop'].sum()
        cum_cases = grouped['case_count'].sum()

        if year >= plot_start_year:
            intercepts.append(intercept)
            intercept_err_lower.append(intercept - intercept_ci_lower)
            intercept_err_upper.append(intercept_ci_upper - intercept)
            betas.append(beta)
            beta_err_lower.append(beta - beta_ci_lower)
            beta_err_upper.append(beta_ci_upper - beta)
            r2_values.append(r2)
            total_populations.append(total_pop)
            cumulative_cases.append(cum_cases)
            years_to_plot.append(year)

            print(
                f"{year}: Intercept = {intercept:.4f} [{intercept_ci_lower:.4f}, {intercept_ci_upper:.4f}], "
                f"β = {beta:.4f} [{beta_ci_lower:.4f}, {beta_ci_upper:.4f}], R² = {r2:.4f}, "
                f"Total Pop = {total_pop}, Cum Cases = {cum_cases}"
            )
    else:
        if year >= plot_start_year:
            intercepts.append(np.nan)
            intercept_err_lower.append(np.nan)
            intercept_err_upper.append(np.nan)
            betas.append(np.nan)
            beta_err_lower.append(np.nan)
            beta_err_upper.append(np.nan)
            r2_values.append(np.nan)
            total_populations.append(np.nan)
            cumulative_cases.append(np.nan)
            years_to_plot.append(year)
            print(f"{year}: insufficient data for regression")

# Plotting the cumulative scaling exponent β
y_err = np.array([beta_err_lower, beta_err_upper])

plt.figure(figsize=(18, 12))
plt.plot(years_to_plot, betas, color='lightseagreen', linewidth=2, label='β-value Estimate')
plt.errorbar(
    years_to_plot,
    betas,
    yerr=y_err,
    fmt='o',
    ecolor='k',
    capsize=8,
    capthick=2,
    color='darkorange',
    elinewidth=2,
    label=r'$\beta$ with 95% CI'
)

plt.title(r'Scaling Exponent ($\beta$) of Total NamUS Missing Person Cases vs County Population (2010–2024)', fontsize=28)
plt.xlabel('Cumulative NamUS Cases by Year', fontsize=24)
plt.ylabel(r'Estimated Scaling Exponent Value ($\beta$)', fontsize=24)
plt.xticks(years_to_plot, rotation=65, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
max_beta = np.nanmax(betas)
plt.ylim(0, max_beta + 0.05)

plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\temporal\[2010-2024]regression_ts_counties.png', dpi=1200, bbox_inches='tight')
plt.show()

# Identify years with best and worst R²
r2_array = np.array(r2_values)
valid_r2_indices = np.where(~np.isnan(r2_array))[0]
best_year_idx = valid_r2_indices[np.argmax(r2_array[valid_r2_indices])]
worst_year_idx = valid_r2_indices[np.argmin(r2_array[valid_r2_indices])]

best_year = years_to_plot[best_year_idx]
worst_year = years_to_plot[worst_year_idx]


def plot_regression_scatter(ax, year, df, title_prefix=''):
    # Use only cases that occurred within the specific year
    df_year = df[df['Year'] == year]

    # Group by spatial units and count cases for that year
    grouped = (
        df_year.groupby(['State', 'FIPS', 'County_pop'])
        .agg(case_count=('CaseID', 'count'))
        .reset_index()
    )

    # Filter valid rows
    grouped = grouped[
        (grouped['County_pop'] > 0) &
        (grouped['case_count'] > 0)
    ]

    # Log-transform
    grouped['log_pop'] = np.log10(grouped['County_pop'])
    grouped['log_cases'] = np.log10(grouped['case_count'])

    # Remove infs and NaNs
    grouped = grouped[
        np.isfinite(grouped['log_pop']) & np.isfinite(grouped['log_cases'])
    ]

    X = grouped['log_pop'].values
    y = grouped['log_cases'].values

    if len(X) < 2:
        ax.set_title(f"{title_prefix}{year}: Not enough data", fontsize=16)
        return

    # Fit regression
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    y_pred = model.predict(X_const)

    # Extract stats
    intercept = model.params[0]
    beta = model.params[1]
    conf_int = model.conf_int()
    intercept_ci_lower, intercept_ci_upper = conf_int[0, 0], conf_int[0, 1]
    beta_ci_lower, beta_ci_upper = conf_int[1, 0], conf_int[1, 1]

    # Plot data and regression
    ax.scatter(X, y, color='steelblue', alpha=0.7, label='Data Points')
    ax.plot(X, y_pred, color='crimson', linewidth=2, label='Regression Line')

    # Confidence interval shading
    X_range = np.linspace(X.min(), X.max(), 200)
    y_lower = intercept + beta_ci_lower * X_range
    y_upper = intercept + beta_ci_upper * X_range
    ax.fill_between(X_range, y_lower, y_upper, color='crimson', alpha=0.2, label='95% CI')

    # Title and formatting
    ax.set_title(f"{title_prefix}{year} (R² = {model.rsquared:.3f})", fontsize=22)
    ax.set_xlabel(r'$\log_{10}$(Population)', fontsize=24)
    ax.set_ylabel(r'$\log_{10}$(Cases)', fontsize=24)
    ax.tick_params(axis='both', labelsize=20)
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    ax.grid(True)

    ax.legend(title=(
        f"β = {beta:.3f}; CI: [{beta_ci_lower:.3f}, {beta_ci_upper:.3f}]\n"
        f"γ = {intercept:.3f}; CI: [{intercept_ci_lower:.3f}, {intercept_ci_upper:.3f}]"
    ), fontsize=14, title_fontsize=16)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))
plot_regression_scatter(axes[0], worst_year, df, title_prefix='Worst Year (Cumulative): ')
plot_regression_scatter(axes[1], best_year, df, title_prefix='Best Year (Cumulative): ')
plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\temporal\[2010-2024]regression_comparison_ts_counties.png', dpi=1200, bbox_inches='tight')
plt.show()

def plot_r2_timeseries(years, r2_values):
    """
    Plots R² values over time as a time series.

    Parameters:
    - years: list of years (2000 to 2024)
    - r2_values: list of R² values corresponding to each year
    """
    r2_array = np.array(r2_values)
    
    plt.figure(figsize=(14, 6))
    plt.plot(years, r2_array, marker='o', color='darkgreen', linewidth=3.5, label=r'$R^2$ value')
    
    # Highlight best and worst years
    if np.any(~np.isnan(r2_array)):
        best_idx = np.nanargmax(r2_array)
        worst_idx = np.nanargmin(r2_array)

        plt.scatter(years[best_idx], r2_array[best_idx], color='darkorange', s=100, label=f'Best R²: {years[best_idx]}')
        plt.scatter(years[worst_idx], r2_array[worst_idx], color='crimson', s=100, label=f'Worst R²: {years[worst_idx]}')

    plt.title(r'Time Series of $R^2$ Values (2010–2024) for $\beta$ for Counties', fontsize=32)
    plt.xlabel('Year', fontsize=28)
    plt.ylabel(r'$R^2$ Value', fontsize=28)
    plt.xticks(years, rotation=65, fontsize=24)
    plt.yticks(fontsize=24)
    plt.ylim(0, 1.05)
    plt.grid(True, alpha=0.6)
    plt.legend(fontsize=18)
    plt.tight_layout()
    plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\temporal\[2010-2024]r2_ts_counties.png', dpi=1200, bbox_inches='tight')
    plt.show()

plot_r2_timeseries(years, r2_array)

##### Scaling of Total NamUs over Select Time Periods vs Population:

In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm

df_primary = pd.read_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv')
df_primary['DisappearanceDate'] = pd.to_datetime(df_primary['DisappearanceDate'])

df_primary = df_primary[
    (df_primary['DisappearanceDate'] > pd.to_datetime('2009-12-31')) &
    (df_primary['DisappearanceDate'] < pd.to_datetime('2025-01-01'))
]

df_msa = df_primary.groupby('MSA Code').agg(
    Case_Count=('CaseID', 'count'),      # count the number of rows per MSA
    MSA_Title=('MSA Title', 'first'),   # take the first MSA Title
    CBSA_Type=('CBSA Type', 'first'),    # take the first CBSA Type (or whatever column you meant by 'MSA Type')
    MSA_pop=('MSA_pop', 'last')
).reset_index()

# --- Clean and prepare data
df_msa = df_msa[(df_msa['Case_Count'] > 0) & (df_msa['MSA_pop'] > 0)].dropna()
df_msa['log_cases'] = np.log10(df_msa['Case_Count'])
df_msa['log_pop'] = np.log10(df_msa['MSA_pop'])

# Subsets
datasets = {
    'All CBSAs': df_msa,
    'MSAs': df_msa[df_msa['CBSA_Type'] == 'MSA'],
    'MicroSAs': df_msa[df_msa['CBSA_Type'] == 'MicroSA'],
}

# --- Plot setup
fig, axes = plt.subplots(1, 3, figsize=(20, 8), sharey=True)

for ax, (title, df) in zip(axes, datasets.items()):
    # Fit log-log regression
    X = sm.add_constant(df['log_pop'])
    y = df['log_cases']
    model = sm.OLS(y, X).fit()

    intercept, slope = model.params
    conf_int = model.conf_int(alpha=0.05)
    intercept_ci = conf_int.loc['const'].values
    slope_ci = conf_int.loc['log_pop'].values
    r2 = model.rsquared

    # Line and confidence interval
    x_vals = np.linspace(0, df['log_pop'].max(), 200)
    x_vals_const = sm.add_constant(x_vals)
    y_vals = model.predict(x_vals_const)
    preds_ci = model.get_prediction(x_vals_const).summary_frame(alpha=0.05)

    # Calculate mean and median points
    mean_log_pop = df['log_pop'].mean()
    mean_log_cases = df['log_cases'].mean()
    median_log_pop = df['log_pop'].median()
    median_log_cases = df['log_cases'].median()

    # Plotting
    ax.scatter(df['log_pop'], df['log_cases'], color='steelblue', alpha=0.7, label='$log_{10}$(Number of Cases per CBSA)')
    ax.plot(x_vals, y_vals, color='darkred', linewidth=2, label='Regression line')
    ax.fill_between(x_vals, preds_ci['mean_ci_lower'], preds_ci['mean_ci_upper'],
                    color='lightcoral', alpha=0.3, label='95% CI band')

    ax.scatter(mean_log_pop, mean_log_cases, color='green', s=100, edgecolor='black', label='Mean point', zorder=5)
    ax.scatter(median_log_pop, median_log_cases, color='purple', s=100, edgecolor='black', label='Median point', zorder=5)

    # Total cases as sum, not count
    total_cases = df['Case_Count'].sum()
    regression_label = (
        f"β = {slope:.3f} [{slope_ci[0]:.3f}, {slope_ci[1]:.3f}]\n"
        f"γ = {intercept:.3f} [{intercept_ci[0]:.3f}, {intercept_ci[1]:.3f}]\n"
        f"$R^2$ = {r2:.3f}\n"
        f"Total cases: {total_cases:,.0f}"  # formatted with commas
    )
    ax.plot([], [], ' ', label=regression_label)

    ax.set_title(title, fontsize=28)
    ax.tick_params(axis='both', labelsize=16)
    ax.legend(fontsize=12, loc='upper left')
    ax.grid(True)
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)

axes[0].set_ylabel('log(NamUS Case Counts)\n[2010–2024]', fontsize=20)
axes[0].set_xlabel('log(CBSA Population)', fontsize=20)
axes[1].set_xlabel('log(MSA Population)', fontsize=20)
axes[2].set_xlabel('log(MicroSA Population)', fontsize=20)

fig.suptitle('Scaling Exponent (β) of Missing Persons Cases vs Population for CBSAs [2010–2024]', fontsize=28)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\[2010-2024]regressions.png', dpi=1200, bbox_inches='tight')

### Demographic Plots:

#### CBSA Type Distribution:

In [ ]:
import pandas as pd  
import matplotlib.pyplot as plt

df_primary = pd.read_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv')
df_primary['DisappearanceDate'] = pd.to_datetime(df_primary['DisappearanceDate'])

df_primary = df_primary[
    (df_primary['DisappearanceDate'] > pd.to_datetime('1968-12-31')) &
    (df_primary['DisappearanceDate'] < pd.to_datetime('2025-01-01'))
]

def plot_cbsa_type_distribution(df):

    if 'CBSA Type' not in df.columns:
        raise ValueError("'CBSA Type' column not found in the DataFrame.")
    
    # Count values, include NaNs under 'None'
    counts = df['CBSA Type'].value_counts(dropna=False)
    counts.index = counts.index.fillna('None')
    total = counts.sum()

    # Convert counts to DataFrame for sorting and indexing
    counts_df = counts.sort_values(ascending=True).to_frame(name='count')
    counts_df['percentage'] = counts_df['count'] / total * 100

    # Use a color map for distinct colors
    cmap = plt.get_cmap('tab20')  # Choose from: 'Set3', 'tab20', 'viridis', etc.
    colors = [cmap(i) for i in range(len(counts_df))]

    # Plot
    plt.figure(figsize=(18, 8))
    bars = plt.bar(
        counts_df.index,
        counts_df['count'],
        color=colors,
        edgecolor='black'
    )

    # Annotate each bar with count and percentage
    for bar, count, perc in zip(bars, counts_df['count'], counts_df['percentage']):
        height = bar.get_height()
        if height > 0:
            plt.text(
                bar.get_x() + bar.get_width() / 2,
                height / 2,  # vertically centered inside the bar
                f'{count:,}\n({perc:.1f}%)',
                ha='center',
                va='center',
                fontsize=16,
                color='black' if height > total * 0.05 else 'coral',  # ensure contrast
                fontweight='bold'
            )


    plt.text(
        0.05, 0.95,
        f"Total Cases: {total:,}",
        transform=plt.gca().transAxes,
        fontsize=22,
        ha='left',
        va='top',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='black')
    )
    # Labels and formatting
    plt.title(
        f"Distribution of NamUS Missing Persons Cases by CBSA Type (1969–2024)",
        fontsize=28
    )
    plt.xlabel("CBSA Type", fontsize=28)
    plt.ylabel("Number of Cases", fontsize=28)
    plt.xticks(rotation=45, fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(r"F:\dsl_CLIMA\projects\submittable\missing persons\plots\type_distribution\[1969-2024]mp_type_distribution(cbsa).png", dpi=1200, bbox_inches='tight')
    plt.show()

plot_cbsa_type_distribution(df_primary)

#### Choropleth Plots:

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# --------------------------------------------------
# Load data
# --------------------------------------------------

df_namus = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv'
)

gdf_2024 = gpd.read_file(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\source\shape files\2024\counties\tl_2024_us_county.shp'
)

gdf_states_2024 = gpd.read_file(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\source\shape files\2024\states\tl_2024_us_state.shp'
)

gdf_2024['GEOID'] = gdf_2024['GEOID'].astype(str)
gdf_2024['STATEFP'] = gdf_2024['STATEFP'].astype(str)
gdf_states_2024['GEOID'] = gdf_states_2024['GEOID'].astype(str)
gdf_states_2024['STATEFP'] = gdf_states_2024['STATEFP'].astype(str)

# --------------------------------------------------
# Keep only continental US (exclude AK, HI, PR)
# --------------------------------------------------

conus_states = {'02', '15', '72'}  # Alaska, Hawaii, Puerto Rico
gdf_2024 = gdf_2024[~gdf_2024['STATEFP'].isin(conus_states)].copy()
gdf_states_2024 = gdf_states_2024[~gdf_states_2024['STATEFP'].isin(conus_states)].copy()

# --------------------------------------------------
# Prepare case counts
# --------------------------------------------------

df_namus['FIPS'] = (
    df_namus['FIPS']
    .astype(str)
    .str.zfill(5)
)

county_counts = (
    df_namus
    .groupby('FIPS')
    .size()
    .reset_index(name='case_count')
)

gdf = gdf_2024.merge(
    county_counts,
    left_on='GEOID',
    right_on='FIPS',
    how='left'
)
gdf['case_count'] = gdf['case_count'].fillna(0)

# --------------------------------------------------
# Log scaling (exclude zeros)
# --------------------------------------------------

vmin = gdf.loc[gdf['case_count'] > 0, 'case_count'].min()
vmax = gdf['case_count'].max()

norm = LogNorm(vmin=vmin, vmax=vmax)

# --------------------------------------------------
# Plot choropleth
# --------------------------------------------------

fig, ax = plt.subplots(figsize=(14, 8))

# Plot map
gdf.plot(
    column='case_count',
    ax=ax,
    cmap='viridis',
    linewidth=0.1,
    edgecolor='gray',
    norm=norm,
    legend=False  # We'll add a horizontal colorbar manually
)

# Zoom to continental US bounds (approximate)
ax.set_xlim([-125, -66])  # longitude
ax.set_ylim([24, 50])     # latitude

# Add horizontal colorbar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm._A = []  # dummy array for ScalarMappable
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.05, pad=0.05)
cbar.set_label('$log_{10}$(Cumulative Missing Person Cases) (1969–2024)', fontsize=12)

ax.set_title(
    "Cumulative NamUs Missing Person Cases by County (Continental U.S., 1969–2024)",
    fontsize=24,
    fontweight='bold'
)

ax.axis('off')

plt.tight_layout()
plt.savefig(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\demographics\[1969-2024]_mp_county_choropleth.png',
    dpi=1200,
    bbox_inches='tight'
)
plt.show()
####################################################################################################################


# --------------------------------------------------
# Prepare case counts (STATE-LEVEL)
# --------------------------------------------------

df_namus['FIPS'] = (
    df_namus['FIPS']
    .astype(str)
    .str.zfill(5)
)

# Extract state FIPS (first 2 digits)
df_namus['STATEFP'] = df_namus['FIPS'].str[:2]

state_counts = (
    df_namus
    .groupby('STATEFP')
    .size()
    .reset_index(name='case_count')
)

gdf = gdf_states_2024.merge(
    state_counts,
    on='STATEFP',
    how='left'
)

gdf['case_count'] = gdf['case_count'].fillna(0)

# --------------------------------------------------
# Log scaling (exclude zeros)
# --------------------------------------------------

vmin = gdf.loc[gdf['case_count'] > 0, 'case_count'].min()
vmax = gdf['case_count'].max()

norm = LogNorm(vmin=vmin, vmax=vmax)

# --------------------------------------------------
# Plot choropleth
# --------------------------------------------------

fig, ax = plt.subplots(figsize=(14, 8))

gdf.plot(
    column='case_count',
    ax=ax,
    cmap='viridis',
    linewidth=0.6,
    edgecolor='gray',
    norm=norm,
    legend=False
)

# Zoom to continental US bounds
ax.set_xlim([-125, -66])
ax.set_ylim([24, 50])

# Horizontal colorbar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm._A = []
cbar = fig.colorbar(
    sm,
    ax=ax,
    orientation='horizontal',
    fraction=0.05,
    pad=0.05
)
cbar.set_label(
    '$log_{10}$(Cumulative Missing Person Cases) (1969–2024)',
    fontsize=12
)

ax.set_title(
    "Cumulative NamUs Missing Person Cases by State "
    "(Continental U.S., 1969–2024)",
    fontsize=24,
    fontweight='bold'
)

ax.axis('off')

plt.tight_layout()
plt.savefig(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\demographics\[1969-2024]_mp_state_choropleth.png',
    dpi=1200,
    bbox_inches='tight'
)
plt.show()

#### Pi Charts:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_namus = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv'
)

df_namus['DisappearanceDate'] = pd.to_datetime(df_namus['DisappearanceDate'])

df_namus = df_namus[
    (df_namus['DisappearanceDate'] > pd.to_datetime('1968-12-31')) &
    (df_namus['DisappearanceDate'] < pd.to_datetime('2025-01-01'))
]

df_namus['Sex'] = df_namus['Sex'].astype(str).str.strip().str.capitalize()
df_namus['Ethnicity'] = df_namus['Ethnicity'].astype(str).str.strip()

df_namus = df_namus.dropna(subset=['Sex', 'Ethnicity'])

####################################################
# Pi Chart of NamUs Missing Persons Cases
####################################################
sex_counts = df_namus['Sex'].value_counts()
n_sex = sex_counts.sum()

fig, ax = plt.subplots(figsize=(8, 6))

ax.pie(
    sex_counts.values,
    labels=sex_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False,
    wedgeprops={'edgecolor': 'white'}
)

ax.set_title(
    f"Sex Distribution of Cumulative NamUs Missing Persons[1969-2024] Cases\n(N = {n_sex:,} cases)",
    fontsize=14,
    fontweight='bold'
)

plt.tight_layout()
plt.savefig(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\demographics\mp_sex_distribution.png',
    dpi=1200,
    bbox_inches='tight'
)
plt.show()

#### Population Pyraminds:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

df_namus = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv'
)

df_namus['DisappearanceDate'] = pd.to_datetime(df_namus['DisappearanceDate'])

df_namus = df_namus[
    (df_namus['DisappearanceDate'] > pd.to_datetime('1968-12-31')) &
    (df_namus['DisappearanceDate'] < pd.to_datetime('2025-01-01'))
]

####################################################
# Population Pyramid of Missing Persons
####################################################

# Keep only rows with valid age + sex
df_plot = df_namus.dropna(subset=['CurrentMinAge', 'CurrentMaxAge', 'Sex']).copy()

# Optional: standardize sex labels just in case
df_plot['Sex'] = df_plot['Sex'].str.capitalize()

# Number of included cases (IMPORTANT: from original rows)
n_cases = df_plot.shape[0]

age_bins = [
    (0, 4), (5, 9), (10, 14), (15, 17), (18, 19), (20, 20), (21, 21),
    (22, 24), (25, 29), (30, 34), (35, 39), (40, 44), (45, 49),
    (50, 54), (55, 59), (60, 61), (62, 64), (65, 66), (67, 69),
    (70, 74), (75, 79), (80, 84), (85, 120)
]

age_labels = [
    'Under 5', '5-9', '10-14', '15-17', '18-19', '20', '21', '22-24',
    '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-61', '62-64', '65-66', '67-69', '70-74', '75-79', '80-84', '85+'
]

def expand_to_age_bins(df):
    records = []

    for _, row in df.iterrows():
        for (low, high), label in zip(age_bins, age_labels):
            if row['CurrentMaxAge'] >= low and row['CurrentMinAge'] <= high:
                records.append({
                    'Sex': row['Sex'],
                    'AgeBin': label
                })

    return pd.DataFrame(records)

expanded = expand_to_age_bins(df_plot)

counts = (
    expanded
    .groupby(['AgeBin', 'Sex'])
    .size()
    .unstack(fill_value=0)
    .reindex(age_labels)
)

# Ensure columns exist even if one sex is missing
counts = counts.reindex(columns=['Male', 'Female'], fill_value=0)

total = counts.sum().sum()
male_percent = counts['Male'] / total * 100
female_percent = counts['Female'] / total * 100

y = np.arange(len(age_labels))

fig, ax = plt.subplots(figsize=(10, 8))

ax.barh(y, -male_percent.values, color='steelblue', label='Male')
ax.barh(y,  female_percent.values, color='lightcoral', label='Female')

ax.set_yticks(y)
ax.set_yticklabels(age_labels)
ax.axvline(0, color='black', linewidth=1)

# Make x-axis labels show positive percentages
ax.xaxis.set_major_formatter(
    mtick.FuncFormatter(lambda x, _: f"{abs(x):.0f}%")
)

# Symmetric x-limits
max_val = max(male_percent.max(), female_percent.max())
ax.set_xlim(-max_val * 1.1, max_val * 1.1)

ax.set_xlabel("Percent of Total (%)")
ax.set_title("Age / Sex Distribution of Cumulative Missing Persons Cases [1969-2024]")

# Add N label (bottom-right)
ax.text(
    0.99, 0.01,
    f"N = {n_cases:,} cases",
    transform=ax.transAxes,
    ha='right',
    va='bottom',
    fontsize=11,
    color='gray'
)

ax.legend()
plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\demographics\population_pyramids\[1969-2024]mp_pop_pyramid.png', dpi=1200, bbox_inches='tight')
plt.show()

#### Bar Charts of Ethnicity/Race Distribution:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_namus = pd.read_csv(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\export\mp_term.csv'
)

df_namus['DisappearanceDate'] = pd.to_datetime(df_namus['DisappearanceDate'])

df_namus = df_namus[
    (df_namus['DisappearanceDate'] > pd.to_datetime('1968-12-31')) &
    (df_namus['DisappearanceDate'] < pd.to_datetime('2025-01-01'))
]

df_namus['Sex'] = df_namus['Sex'].astype(str).str.strip().str.capitalize()
df_namus['Ethnicity'] = df_namus['Ethnicity'].astype(str).str.strip()

df_namus = df_namus.dropna(subset=['Sex', 'Ethnicity'])


# ==================================================
# BAR CHART: Ethnicity Distribution
# ==================================================

eth_counts = df_namus['Ethnicity'].value_counts()
n_eth = eth_counts.sum()

# Group small categories
threshold = 0.03  # 3%
eth_percent = eth_counts / n_eth
small = eth_percent < threshold

eth_counts_grouped = eth_counts.copy()
if small.any():
    eth_counts_grouped = eth_counts_grouped[~small]
    eth_counts_grouped['Other'] = eth_counts[small].sum()

# Convert to percentages
eth_percent_grouped = eth_counts_grouped / n_eth * 100

# Sort for nicer plotting
eth_percent_grouped = eth_percent_grouped.sort_values()

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.barh(
    eth_percent_grouped.index,
    eth_percent_grouped.values,
    color='slategray',
    edgecolor='black',
    alpha=0.85
)

# Label bars with % and count
for bar, label in zip(bars, eth_percent_grouped.index):
    pct = eth_percent_grouped[label]
    count = eth_counts_grouped[label]
    ax.text(
        bar.get_width() + 0.3,
        bar.get_y() + bar.get_height() / 2,
        f"{pct:.1f}% (N={count:,})",
        va='center',
        fontsize=10
    )

ax.set_xlabel("Percent of Total (%)")
ax.set_title(
    f"Ethnicity Distribution of Cumulative NamUs Missing Persons[1969-2024] Cases\n(N = {n_eth:,} cases)",
    fontsize=14,
    fontweight='bold'
)

ax.set_xlim(0, eth_percent_grouped.max() * 1.25)

plt.tight_layout()
plt.savefig(
    r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\regressions\demographics\mp_ethnicity_bar.png',
    dpi=1200,
    bbox_inches='tight'
)
plt.show()

## Mexico INEGI Visuals:

### Scaling per 100k [2006-2025]:

In [ ]:
"""

This script creates a bar chart showing the trend of the rate of
missing people for the specified state in Mexico.

It is very easy to use, you only need to run the main function with
one parameter: the ID of the state.

The missing people data was sourced from:

https://consultapublicarnpdno.segob.gob.mx/consulta

The original data was cleaned, translated and anonymized.

The population data was sourced from:

https://datos.gob.mx/busca/dataset/proyecciones-de-la-poblacion-de-mexico-y-de-las-entidades-federativas-2020-2070/resource/ae83f2b0-f23e-45e3-91ae-f85594775dff

The population dataset included in this project is meant to only
be used within the project, as it had some adjustments done (removal of
columns and translation).

This project uses the free Montserrat font -> https://fonts.google.com/specimen/Montserrat

"""

import pandas as pd
import plotly.graph_objects as go


# Each state_id has a corresponding name.
# 0 is for national level figures.
STATES = {
    0: "Mexico",
    1: "Aguascalientes",
    2: "Baja California",
    3: "Baja California Sur",
    4: "Campeche",
    5: "Coahuila",
    6: "Colima",
    7: "Chiapas",
    8: "Chihuahua",
    9: "Ciudad de México",
    10: "Durango",
    11: "Guanajuato",
    12: "Guerrero",
    13: "Hidalgo",
    14: "Jalisco",
    15: "Estado de México",
    16: "Michoacán",
    17: "Morelos",
    18: "Nayarit",
    19: "Nuevo León",
    20: "Oaxaca",
    21: "Puebla",
    22: "Querétaro",
    23: "Quintana Roo",
    24: "San Luis Potosí",
    25: "Sinaloa",
    26: "Sonora",
    27: "Tabasco",
    28: "Tamaulipas",
    29: "Tlaxcala",
    30: "Veracruz",
    31: "Yucatán",
    32: "Zacatecas",
}


def main(state_id):
    """
    Creates a bar chart with the yearly rate of missing people
    for the specified state.

    Parameters
    ----------
    state_id : int
        The ID of the state you want to plot.
        Use 0 for national figures.

    """

    # We load the population dataset.
    pop = pd.read_csv(r"F:\dsl_CLIMA\projects\submittable\missing persons\source\mexico_missing_persons\population.csv")

    # We filter by the specified state_id.
    pop = pop[pop["STATE_ID"] == state_id]

    # We calculate the total population by year.
    pop = pop.groupby("YEAR").sum(numeric_only=True)

    # We load the missing people dataset.
    df = pd.read_csv(r"F:\dsl_CLIMA\projects\submittable\missing persons\source\mexico_missing_persons\data.csv")

    # A victim can have multiple reports, the first thing to do
    # is to only select one record per victim.
    df = df.groupby("VICTIM_ID").last()

    # We filter by state. If it is 0 we skip this step as
    # 0 is for national level figures.
    if state_id != 0:
        df = df[df["STATE_ID"] == state_id]

    # We will convert to datetime the date of incidence and date of report columns.
    df["DATE_OF_INCIDENCE"] = pd.to_datetime(df["DATE_OF_INCIDENCE"], errors="coerce")
    df["DATE_OF_REPORT"] = pd.to_datetime(df["DATE_OF_REPORT"], errors="coerce")

    # To get the counts by year we will prefer the date of incidence
    # but we don't always have it. In that case we fallback to the date of report.
    df["DATE_OF_INCIDENCE"] = df["DATE_OF_INCIDENCE"].fillna(df["DATE_OF_REPORT"])

    # Now we calculate the yearly counts.
    df = df["DATE_OF_INCIDENCE"].value_counts().resample("YS").sum().to_frame("total")

    # We only need the year from the index.
    df.index = df.index.year

    # We add the population to our missing people DataFrame.
    df["pop"] = pop["POPULATION"]

    # We calculate the rate per 100,000 inhabitants.
    df["rate"] = df["total"] / df["pop"] * 100000

    # We only select the latest 20 years.
    df = df.tail(20)

    # We create the text for each bar.
    df["text"] = df.apply(
        lambda x: f"<b>{x['rate']:,.2f}</b><br>({x['total']:,.0f})", axis=1
    )

    # We will create a simple bar chart with all the previous calculations.
    # The bar chart will have a color scale from yellow (0) to red (max).
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df["rate"],
            text=df["text"],
            name=f"Cummulative total: <b>{df['total'].sum():,.0f}</b> actively missing.<br>Doesn't include confidential records.",
            textposition="outside",
            marker_color=df["rate"],
            marker_colorscale="portland",
            marker_cmid=0,
            marker_line_width=0,
            textfont_size=30,
        )
    )

    fig.update_xaxes(
        ticks="outside",
        ticklen=10,
        zeroline=False,
        tickcolor="#FFFFFF",
        linewidth=2,
        showline=True,
        showgrid=True,
        gridwidth=0.35,
        mirror=True,
        nticks=len(df) + 1,
    )

    # The y-axis range is dinamically set to make room for the tallest bar text.
    fig.update_yaxes(
        title="Rate per 100,000 inhabitants",
        range=[0, df["rate"].max() * 1.1],
        ticks="outside",
        separatethousands=True,
        tickfont_size=14,
        ticklen=10,
        title_standoff=15,
        tickcolor="#FFFFFF",
        linewidth=2,
        gridwidth=0.35,
        showline=True,
        nticks=20,
        zeroline=False,
        mirror=True,
    )

    fig.update_layout(
        showlegend=True,
        legend_borderwidth=1,
        legend_bordercolor="#FFFFFF",
        legend_x=0.01,
        legend_y=0.98,
        legend_xanchor="left",
        legend_yanchor="top",
        width=1920,
        height=1080,
        font_family="Montserrat",
        font_color="#FFFFFF",
        font_size=24,
        title_text=f"Evolution of the rate of missing and unaccounted-for people in <b>{STATES[state_id]}</b> ({df.index.min()}-{df.index.max()})",
        title_x=0.5,
        title_y=0.965,
        margin_t=80,
        margin_r=40,
        margin_b=120,
        margin_l=130,
        title_font_size=34,
        paper_bgcolor="#2B2B2B",
        plot_bgcolor="#171010",
        annotations=[
            dict(
                x=0.01,
                y=-0.11,
                xref="paper",
                yref="paper",
                xanchor="left",
                yanchor="top",
                text="Source: RNPDNO (July 2025)",
            ),
            dict(
                x=0.5,
                y=-0.11,
                xref="paper",
                yref="paper",
                xanchor="center",
                yanchor="top",
                text="Year of incidence",
            ),
            dict(
                x=1.01,
                y=-0.11,
                xref="paper",
                yref="paper",
                xanchor="right",
                yanchor="top",
                text="🧁 @lapanquecita",
            ),
        ],
    )

    # We name the resulting figure with the state_id.
    fig.write_image(fr"F:\dsl_CLIMA\projects\submittable\missing persons\plots\mexico\{state_id}.png")


if __name__ == "__main__":
    main(0)

### INEGI Age & Sex Demographic Plots:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Example: df already exists
df_inegi = pd.read_csv(r'F:\dsl_CLIMA\projects\submittable\missing persons\source\mexico_missing_persons\data.csv', dtype=str)

# Ensure DATE_OF_BIRTH is datetime
df_inegi["DATE_OF_BIRTH"] = pd.to_datetime(df_inegi["DATE_OF_BIRTH"], errors="coerce")
df_inegi["DATE_OF_INCIDENCE"] = pd.to_datetime(df_inegi["DATE_OF_INCIDENCE"], errors="coerce")

# -----------------------
# PIE CHART: SEX
# -----------------------
# Normalize SEX values and keep special categories
df_inegi["SEX_CLEAN"] = (
    df_inegi["SEX"]
    .fillna("MISSING")
    .str.upper()
)

# Count values
sex_counts = df_inegi["SEX_CLEAN"].value_counts()

# Desired order
desired_order = ["MISSING", "CONFIDENTIAL", "MALE", "FEMALE"]

# Reindex to enforce order and keep existing categories
sex_counts = sex_counts.reindex(
    [x for x in desired_order if x in sex_counts.index]
)

plt.figure(figsize=(7, 7))

def autopct_with_counts(values):
    def inner(pct):
        total = sum(values)
        count = int(round(pct * total / 100.0))
        return f"{pct:.1f}%\n(n={count})"
    return inner

wedges, _, _ = plt.pie(
    sex_counts,
    autopct=autopct_with_counts(sex_counts),
    startangle=0  # x = 0
)

plt.title("Sex Distribution of INEGI Missing Persons Cases")

# Legend at the bottom with counts
legend_labels = [f"{sex} (n={count})" for sex, count in sex_counts.items()]
plt.legend(
    wedges,
    legend_labels,
    title="Sex",
    loc="lower center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=2
)

plt.axis("equal")
plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\mexico\INEGI_sex_pi_chart.png')
plt.show()

# -----------------------
# BAR CHART: AGE BRACKETS
# -----------------------
today = pd.Timestamp(datetime.today())
df_inegi["AGE_MISSING"] = ((df_inegi['DATE_OF_INCIDENCE'] - df_inegi["DATE_OF_BIRTH"]).dt.days // 365)

bins = [0, 10, 20, 30, 40, 50, 60, 70, 200]
labels = ["0–9", "10–19", "20–29", "30–39",
          "40–49", "50–59", "60–69", "70+"]

df_inegi["AGE_BRACKET"] = pd.cut(df_inegi["AGE_MISSING"], bins=bins, labels=labels, right=False)

age_counts = df_inegi["AGE_BRACKET"].value_counts().sort_index()

plt.figure(figsize=(10, 6))
bars = plt.bar(age_counts.index.astype(str), age_counts.values)

plt.xlabel("Age Bracket")
plt.ylabel("Number of Cases")
plt.title("Distribution of INEGI Missing Persons Cases: Ages at Incidence")

# Add count labels above bars
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{int(height)}",
        ha="center",
        va="bottom"
    )

plt.tight_layout()
plt.savefig(r'F:\dsl_CLIMA\projects\submittable\missing persons\plots\mexico\INEGI_age_at_incidence_barChart.png')
plt.show()

# Missing Persons & Population Data Sources

---

## United States Missing Persons & Population Data

### Census & Population Data
- **University of Brown’s Guide to Census Data**  
  https://libguides.brown.edu/census/histmicro

- **US Census Population Estimates by County (1969–2022)**  
  https://seer.cancer.gov/popdata.thru.2022/download.html

- **SEER FIPS Updates (1969–2022)**  
  https://seer.cancer.gov/seerstat/variables/countyattribs/time-dependent.html

- **David Dorn FIPS Updates (1980–2021)**  
  https://www.ddorn.net/data/FIPS_County_Code_Changes.pdf

- **NHGIS – Historical Shapefiles**  
  https://data2.nhgis.org/main

---

### Missing Persons Scrapers & Tools

#### NightOwlRecon (NamUS Data)
- **GitHub Repository**  
  https://github.com/NightOwlRecon/NamUs-Data/blob/main/namus.py

- **Extracted `.json` Dataset**  
  https://drive.google.com/file/d/1k8PRzRlwE_Ti52enW4qjNX0bInr5Pf0g/view?usp=sharing

#### Prepager (NamUS Scraper)
- **GitHub Repository**  
  https://github.com/Prepager/namus-scraper

---

### Crosswalks & Spatial References
- **Historical County / MSA / CSA Crosswalks (BLS)**  
  https://www.bls.gov/cew/classifications/areas/county-msa-csa-crosswalk.htm

- **Connecticut Town Crosswalks (2023–Present)**  
  https://data.ct.gov/Local-Government/Connecticut-Towns-Crosswalk-with-Tax-Codes-and-FIP/5hqs-h5c3/about_data

- **Connecticut FIPS Codes for Planning Regions (AP Elections API)**  
  https://developer.ap.org/ap-elections-api/docs/CT_FIPS_Codes_forPlanningRegions.htm

---

## South American & Latin American Missing Persons Data

### Mexico
- **INEGI Missing & Unaccounted-for Persons Dataset**  
  https://figshare.com/articles/dataset/Missing_and_Unaccounted-for_People_in_Mexico_1960s_2025_/28283000

- **Governmental Crime & Missing Persons Report**  
  https://www.gob.mx/sesnsp/acciones-y-programas/incidencia-delictiva-del-fuero-comun-nueva-metodologia

### New Mexico
- **INEGI Missing Persons (ELCRI)**  
  https://elcri.men/en/about/

### Colombia
- **SIEDCO Missing Persons Statistics (National Police)**  
  https://www.policia.gov.co/estadistica-delictiva

### Argentina
- **Personas Perdidas**  
  http://personasperdidas.org.ar/looking_for_their_families

- **Police Reports (2020–2024)**  
  https://www.datos.gob.ar/dataset/justicia-lucha-contra-trata-personas---llamados-linea-145---denuncias/archivo/justicia_4b786057-973f-4bd6-9594-8b74233ad9b1

- **Federal System for Searching for Missing and Lost Persons – Management Report**  
  https://www.argentina.gob.ar/sites/default/files/ministerio-seguridad-argentina-informe-gestion-sifebu-2024.pdf

### Brazil
- **Missing Persons & Forced Disappearances (Academic Article)**  
  https://www.sciencedirect.com/science/article/pii/S2665910722000330

---

## Anecdotal / Qualitative Data & Additional Resources

- **The Charley Project – Geographic Case Search (Mexico)**  
  https://charleyproject.org/case-searches/geographical-cases?region=Mexico

- **The Doe Network**  
  https://www.doenetwork.org/

- **The Lost People – NamUS Choropleth Mapping**  
  https://jseibel55.github.io/The-Lost-People/#collapseThree

- **A Consistent County-Level Spatial Crosswalk Since 1790**  
  https://fpeckert.me/papers/egp-spatialcrosswalk.pdf
